In [ ]:
"""
Inventory Management System
A simplified system to manage products, orders, and transactions
"""

import datetime
from collections import deque
import heapq

# Inventory data stored as a list of products
# Each product is a list: [product_id, name, quantity, price]
inventory = [
    [1, "Laptop", 15, 999.99],
    [2, "Mouse", 42, 24.99],
    [3, "Keyboard", 35, 49.99],
    [4, "Monitor", 18, 249.99],
    [5, "Headphones", 56, 79.99]
]

# Composite products (bundles made from multiple products)
# Format: {product_id: {"name": bundle_name, "parts": [(part_id, quantity)]}}
composite_products = {
    6: {
        "name": "Gaming Bundle",
        "parts": [(1, 1), (2, 1), (4, 1), (5, 1)]  # (product_id, quantity)
    },
    7: {
        "name": "Office Setup",
        "parts": [(1, 1), (2, 1), (3, 1)]
    }
}

# Transaction ledger using linked list to record all inventory changes
class TransactionNode:
    """Node to store a single transaction"""
    def __init__(self, transaction_id, transaction_type, product_id, quantity, date):
        self.transaction_id = transaction_id
        self.transaction_type = transaction_type  # ADD, RESTOCK, SALE
        self.product_id = product_id
        self.quantity = quantity
        self.date = date
        self.next = None  # Pointer to next transaction

class TransactionLedger:
    """Linked list to store all transactions"""
    def __init__(self):
        self.head = None  # First transaction
        self.transaction_count = 0  # Total transactions
    
    def add_transaction(self, transaction_type, product_id, quantity):
        """Add a new transaction to the ledger"""
        self.transaction_count += 1
        # Create new transaction node
        new_node = TransactionNode(
            self.transaction_count,
            transaction_type,
            product_id,
            quantity,
            datetime.datetime.now()
        )
        
        # Add to end of linked list
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node
    
    def display_ledger(self):
        """Display all transactions in the ledger"""
        if self.head is None:
            print("No transactions recorded.")
            return
        
        current = self.head
        print("\n--- TRANSACTION LEDGER ---")
        print("ID | Type | Product ID | Quantity | Date")
        print("-" * 50)
        while current:
            print(f"{current.transaction_id} | {current.transaction_type} | {current.product_id} | {current.quantity} | {current.date.strftime('%Y-%m-%d %H:%M')}")
            current = current.next

# Order management classes and queues
class Order:
    """Class to represent a customer order"""
    def __init__(self, order_id, product_id, quantity, customer_name, priority=0):
        self.order_id = order_id
        self.product_id = product_id
        self.quantity = quantity
        self.customer_name = customer_name
        self.priority = priority  # For priority queue (0=regular, 1=high, 2=medium, 3=low)
        self.date_created = datetime.datetime.now()
    
    # Method to compare orders for priority queue
    def __lt__(self, other):
        return self.priority < other.priority

# Order management queues
order_queue = deque()  # Regular FIFO queue for orders
priority_queue = []    # Priority queue for backorders (using heapq)
loading_stack = []     # LIFO stack for loading orders onto truck

order_counter = 1  # Global order ID counter
ledger = TransactionLedger()  # Create transaction ledger

# Helper functions for product operations
def get_product_name(product_id):
    """Get product name by ID from inventory or composite products"""
    # Check regular inventory
    for item in inventory:
        if item[0] == product_id:
            return item[1]
    
    # Check composite products
    if product_id in composite_products:
        return composite_products[product_id]["name"]
    
    return "Unknown Product"

def get_product_price(product_id):
    """Get product price by ID from inventory or calculate for composite products"""
    # Check regular inventory
    for item in inventory:
        if item[0] == product_id:
            return item[3]
    
    # For composite products, calculate total cost
    if product_id in composite_products:
        return calculate_composite_cost(product_id)
    
    return 0  # Default if product not found

def get_product_stock(product_id):
    """Get current stock quantity for a product"""
    for item in inventory:
        if item[0] == product_id:
            return item[2]
    return 0  # Product not found

def update_product_stock(product_id, quantity):
    """Update stock quantity for a product"""
    for item in inventory:
        if item[0] == product_id:
            item[2] += quantity
            return True
    return False  # Product not found

def find_product_by_id(product_id):
    """Find a product by ID in inventory"""
    for item in inventory:
        if item[0] == product_id:
            return item
    return None  # Product not found

# Composite product cost calculation
def calculate_composite_cost(product_id, depth=0):
    """Recursively calculate cost of a composite product"""
    if product_id in composite_products:
        total_cost = 0
        indent = "  " * depth  # For indentation in output
        
        print(f"{indent}Calculating cost for {composite_products[product_id]['name']}:")
        
        # Calculate cost for each part
        for part_id, quantity in composite_products[product_id]["parts"]:
            # Recursive call if part is also a composite product
            if part_id in composite_products:
                part_cost = calculate_composite_cost(part_id, depth + 1)
            else:
                part_cost = get_product_price(part_id)
            
            total_cost += part_cost * quantity
            print(f"{indent}  {quantity} x {get_product_name(part_id)} @ ${part_cost:.2f} each")
        
        print(f"{indent}Total for {composite_products[product_id]['name']}: ${total_cost:.2f}")
        return total_cost
    else:
        # Not a composite product, return regular price
        return get_product_price(product_id)

# Inventory reporting
def generate_inventory_report():
    """Generate a comprehensive inventory report"""
    print("\n--- INVENTORY REPORT ---")
    print("ID | Name | Quantity | Price | Value")
    print("-" * 50)
    total_value = 0
    
    # Display each product
    for item in inventory:
        value = item[2] * item[3]  # quantity * price
        total_value += value
        print(f"{item[0]} | {item[1]} | {item[2]} | ${item[3]:.2f} | ${value:.2f}")
    
    print("-" * 50)
    print(f"Total Inventory Value: ${total_value:.2f}")
    
    # Display composite products
    print("\n--- COMPOSITE PRODUCTS ---")
    for product_id, details in composite_products.items():
        cost = calculate_composite_cost(product_id)
        print(f"{product_id} | {details['name']} | ${cost:.2f}")

# User interface functions
def add_product():
    """Add a new product to inventory"""
    print("\n--- ADD NEW PRODUCT ---")
    name = input("Enter product name: ")
    
    try:
        quantity = int(input("Enter initial quantity: "))
        price = float(input("Enter price: "))
    except ValueError:
        print("Invalid input. Please enter numbers for quantity and price.")
        return
    
    # Generate new product ID (max existing ID + 1)
    new_id = max([item[0] for item in inventory] + [pid for pid in composite_products.keys()]) + 1
    
    # Add to inventory
    inventory.append([new_id, name, quantity, price])
    # Record transaction
    ledger.add_transaction("ADD", new_id, quantity)
    print(f"Product '{name}' added with ID {new_id}")

def update_stock():
    """Update stock quantity for a product"""
    print("\n--- UPDATE STOCK ---")
    try:
        product_id = int(input("Enter product ID: "))
        quantity = int(input("Enter quantity to add (use negative to remove): "))
    except ValueError:
        print("Invalid input. Please enter numbers for ID and quantity.")
        return
    
    # Check if product exists
    product = find_product_by_id(product_id)
    if not product:
        print("Product not found!")
        return
    
    # Check if we have enough stock to remove
    if product[2] + quantity < 0:
        print(f"Cannot remove {abs(quantity)} items. Only {product[2]} available.")
        return
    
    # Update stock
    product[2] += quantity
    # Record transaction
    transaction_type = "RESTOCK" if quantity > 0 else "SALE"
    ledger.add_transaction(transaction_type, product_id, abs(quantity))
    print(f"Stock updated. New quantity: {product[2]}")

def place_order():
    """Place a customer order"""
    global order_counter
    
    print("\n--- PLACE ORDER ---")
    
    try:
        product_id = int(input("Enter product ID: "))
        quantity = int(input("Enter quantity: "))
        customer_name = input("Enter customer name: ")
    except ValueError:
        print("Invalid input. Please enter numbers for ID and quantity.")
        return
    
    # Check if product exists
    product = find_product_by_id(product_id)
    if not product:
        print("Product not found!")
        return
    
    # Check stock availability
    if product[2] >= quantity:
        # Enough stock - process immediately
        product[2] -= quantity
        ledger.add_transaction("SALE", product_id, quantity)
        loading_stack.append(Order(order_counter, product_id, quantity, customer_name))
        print(f"Order {order_counter} placed successfully and added to loading queue.")
        order_counter += 1
    else:
        # Not enough stock - create backorder
        print(f"Insufficient stock. Only {product[2]} available. Creating backorder.")
        priority = int(input("Set priority (1=High, 2=Medium, 3=Low): "))
        heapq.heappush(priority_queue, (priority, Order(order_counter, product_id, quantity, customer_name, priority)))
        order_queue.append(Order(order_counter, product_id, quantity, customer_name))
        print(f"Backorder {order_counter} created with priority {priority}.")
        order_counter += 1

def process_backorders():
    """Process backorders when stock becomes available"""
    print("\n--- PROCESS BACKORDERS ---")
    
    if not priority_queue:
        print("No backorders to process.")
        return
    
    # Process backorders in priority order
    temp_list = []
    while priority_queue:
        priority, order = heapq.heappop(priority_queue)
        temp_list.append((priority, order))
        
        product = find_product_by_id(order.product_id)
        if product and product[2] >= order.quantity:
            # Enough stock now - fulfill order
            product[2] -= order.quantity
            ledger.add_transaction("SALE", order.product_id, order.quantity)
            loading_stack.append(order)
            print(f"Processed backorder {order.order_id} for {order.customer_name}")
        else:
            # Still not enough stock, keep in priority queue
            heapq.heappush(priority_queue, (priority, order))
    
    # Report on unfulfilled orders
    if priority_queue:
        print(f"{len(priority_queue)} backorders could not be fulfilled due to insufficient stock.")

def load_truck():
    """Load orders onto truck (LIFO)"""
    print("\n--- LOADING TRUCK (LIFO) ---")
    
    if not loading_stack:
        print("No orders to load.")
        return
    
    loaded_orders = []
    while loading_stack:
        order = loading_stack.pop()
        loaded_orders.append(order)
        print(f"Loaded order {order.order_id} for {order.customer_name}: {order.quantity} x {get_product_name(order.product_id)}")
    
    print(f"Truck loaded with {len(loaded_orders)} orders.")

def view_orders():
    """Display all orders in different queues"""
    print("\n--- ORDER QUEUES ---")
    
    # Regular orders (FIFO)
    print("Regular Orders (FIFO):")
    if order_queue:
        for order in list(order_queue):
            print(f"Order {order.order_id}: {order.quantity} x {get_product_name(order.product_id)} for {order.customer_name}")
    else:
        print("No regular orders.")
    
    # Backorders (Priority)
    print("\nBackorders (Priority):")
    if priority_queue:
        # Create a copy to avoid modifying the original
        temp_queue = priority_queue.copy()
        heapq.heapify(temp_queue)
        
        while temp_queue:
            priority, order = heapq.heappop(temp_queue)
            print(f"Order {order.order_id} (Priority {priority}): {order.quantity} x {get_product_name(order.product_id)} for {order.customer_name}")
    else:
        print("No backorders.")
    
    # Orders ready for loading (LIFO)
    print("\nOrders Ready for Loading (LIFO):")
    if loading_stack:
        for order in reversed(loading_stack):
            print(f"Order {order.order_id}: {order.quantity} x {get_product_name(order.product_id)} for {order.customer_name}")
    else:
        print("No orders ready for loading.")

# Main menu and program flow
def main():
    """Main program loop with user menu"""
    while True:
        print("\n=== INVENTORY MANAGEMENT SYSTEM ===")
        print("1. View Inventory Report")
        print("2. Add New Product")
        print("3. Update Stock Level")
        print("4. Place Customer Order")
        print("5. Process Backorders")
        print("6. Load Delivery Truck")
        print("7. View Orders")
        print("8. Calculate Composite Product Cost")
        print("9. View Transaction Ledger")
        print("0. Exit")
        
        choice = input("Enter your choice: ")
        
        if choice == "1":
            generate_inventory_report()
        elif choice == "2":
            add_product()
        elif choice == "3":
            update_stock()
        elif choice == "4":
            place_order()
        elif choice == "5":
            process_backorders()
        elif choice == "6":
            load_truck()
        elif choice == "7":
            view_orders()
        elif choice == "8":
            try:
                product_id = int(input("Enter composite product ID: "))
                if product_id in composite_products:
                    cost = calculate_composite_cost(product_id)
                    print(f"Total cost for {composite_products[product_id]['name']}: ${cost:.2f}")
                else:
                    print("Not a valid composite product ID.")
            except ValueError:
                print("Please enter a valid number.")
        elif choice == "9":
            ledger.display_ledger()
        elif choice == "0":
            print("Exiting system. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

# Start the program
if __name__ == "__main__":
    main()


=== INVENTORY MANAGEMENT SYSTEM ===
1. View Inventory Report
2. Add New Product
3. Update Stock Level
4. Place Customer Order
5. Process Backorders
6. Load Delivery Truck
7. View Orders
8. Calculate Composite Product Cost
9. View Transaction Ledger
0. Exit


Enter your choice:  1



--- INVENTORY REPORT ---
ID | Name | Quantity | Price | Value
--------------------------------------------------
1 | Laptop | 15 | $999.99 | $14999.85
2 | Mouse | 42 | $24.99 | $1049.58
3 | Keyboard | 35 | $49.99 | $1749.65
4 | Monitor | 18 | $249.99 | $4499.82
5 | Headphones | 56 | $79.99 | $4479.44
--------------------------------------------------
Total Inventory Value: $26778.34

--- COMPOSITE PRODUCTS ---
Calculating cost for Gaming Bundle:
  1 x Laptop @ $999.99 each
  1 x Mouse @ $24.99 each
  1 x Monitor @ $249.99 each
  1 x Headphones @ $79.99 each
Total for Gaming Bundle: $1354.96
6 | Gaming Bundle | $1354.96
Calculating cost for Office Setup:
  1 x Laptop @ $999.99 each
  1 x Mouse @ $24.99 each
  1 x Keyboard @ $49.99 each
Total for Office Setup: $1074.97
7 | Office Setup | $1074.97

=== INVENTORY MANAGEMENT SYSTEM ===
1. View Inventory Report
2. Add New Product
3. Update Stock Level
4. Place Customer Order
5. Process Backorders
6. Load Delivery Truck
7. View Orders
8. Cal

Enter your choice:  2



--- ADD NEW PRODUCT ---


...


--- ORDER QUEUES ---
Regular Orders (FIFO):
No regular orders.

Backorders (Priority):
No backorders.

Orders Ready for Loading (LIFO):
No orders ready for loading.

=== INVENTORY MANAGEMENT SYSTEM ===
1. View Inventory Report
2. Add New Product
3. Update Stock Level
4. Place Customer Order
5. Process Backorders
6. Load Delivery Truck
7. View Orders
8. Calculate Composite Product Cost
9. View Transaction Ledger
0. Exit


Enter your choice:  8
Enter composite product ID:  6


Calculating cost for Gaming Bundle:
  1 x Laptop @ $999.99 each
  1 x Mouse @ $24.99 each
  1 x Monitor @ $249.99 each
  1 x Headphones @ $79.99 each
Total for Gaming Bundle: $1354.96
Total cost for Gaming Bundle: $1354.96

=== INVENTORY MANAGEMENT SYSTEM ===
1. View Inventory Report
2. Add New Product
3. Update Stock Level
4. Place Customer Order
5. Process Backorders
6. Load Delivery Truck
7. View Orders
8. Calculate Composite Product Cost
9. View Transaction Ledger
0. Exit


Enter your choice:  9



--- TRANSACTION LEDGER ---
ID | Type | Product ID | Quantity | Date
--------------------------------------------------
1 | ADD | 8 | 7 | 2025-08-27 22:02
2 | SALE | 4 | 12 | 2025-08-27 22:03
3 | SALE | 5 | 5 | 2025-08-27 22:06

=== INVENTORY MANAGEMENT SYSTEM ===
1. View Inventory Report
2. Add New Product
3. Update Stock Level
4. Place Customer Order
5. Process Backorders
6. Load Delivery Truck
7. View Orders
8. Calculate Composite Product Cost
9. View Transaction Ledger
0. Exit
